<a href="https://colab.research.google.com/github/jailsonnt/imersao_gemini_alura_google/blob/main/final_imersao_gemini_alura_google_somente_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Instala dependencia

In [ ]:
!pip install -U -q google-generativeai
!pip install -U -q requests
!pip install -U -q regex

Faz importações necessárias

In [16]:
import google.generativeai as genai
import requests
import json
import re

Autentica a API

In [ ]:
print("Informe chave válida da API, a aplicação não irá funcionar se o valor inserido não for válido.")
GOOGLE_API_KEY = input("Insira a chave da API: ")
genai.configure(api_key=GOOGLE_API_KEY)
print(f"chave {GOOGLE_API_KEY} definida")

Definições e configurações do modelo

In [18]:
model_name ="gemini-1.5-pro-latest"
generation_config = {
  "temperature": 1
}

safety_settings = [
  {
    "category": "HARM_CATEGORY_HARASSMENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_HATE_SPEECH",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_SEXUALLY_EXPLICIT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
  {
    "category": "HARM_CATEGORY_DANGEROUS_CONTENT",
    "threshold": "BLOCK_MEDIUM_AND_ABOVE"
  },
]

Definição do prompt principal

In [19]:
prompt = "Você é um assistente virtual, seu nome é \"Iamiga\" com o objetivo de identificar tipos de estabelecimento de interesse para o usúario no momento, após algumas perguntas você irá responder no formato lista de json no formato especificado em \"Modelo resposta\", converse com o usuário para conhecer os gostos do usuario oque quer e após isso sugira o tipo de estabelecimento para buscar no google. Para sugerir a experiencia é importante perguntar se o usúario está sozinho ou acompanhado. Descubra também o endereço aproximado do usuário, descubra no mínimo a cidade mas deixe claro que as sugestões serão melhores se o endereço for mais exato, esse endereço será utilizado na api places do google. Todas as respostas precisam seguir o formato do \"Modelo resposta\", nesse modelo a resposta que encaminharei para o usuário em formato de texto simples deve estar no campo, \"proxima_mensagem\", a partir do momento que houver sugestoes de locais preencha o campo \"buscas\".\n\nModelo resposta:\n{\"response\":{\"proxima_mensagem\":\"resposta da AI\", \"endereco\": \"endereco aproximado\", \"buscas\": [{\"tipo_google\": \"lawyer\", \"tipo\":\"advogado criminalista\" }, {\"tipo_google\": \"bar\", \"tipo\":\"boate\" }]}\n\nNesse modelo o \"tipo_google\" precisa ser um ítem de \"tipo_google_list\"\ntipo_google_list: [\n  \"accounting\",\n  \"airport\",\n  \"amusement_park\",\n  \"aquarium\",\n  \"art_gallery\",\n  \"atm\",\n  \"bakery\",\n  \"bank\",\n  \"bar\",\n  \"beauty_salon\",\n  \"bicycle_store\",\n  \"book_store\",\n  \"bowling_alley\",\n  \"bus_station\",\n  \"cafe\",\n  \"campground\",\n  \"car_dealer\",\n  \"car_rental\",\n  \"car_repair\",\n  \"car_wash\",\n  \"casino\",\n  \"cemetery\",\n  \"church\",\n  \"city_hall\",\n  \"clothing_store\",\n  \"convenience_store\",\n  \"courthouse\",\n  \"dentist\",\n  \"department_store\",\n  \"doctor\",\n  \"drugstore\",\n  \"electrician\",\n  \"electronics_store\",\n  \"embassy\",\n  \"fire_station\",\n  \"florist\",\n  \"funeral_home\",\n  \"furniture_store\",\n  \"gas_station\",\n  \"gym\",\n  \"hair_care\",\n  \"hardware_store\",\n  \"hindu_temple\",\n  \"home_goods_store\",\n  \"hospital\",\n  \"insurance_agency\",\n  \"jewelry_store\",\n  \"laundry\",\n  \"lawyer\",\n  \"library\",\n  \"light_rail_station\",\n  \"liquor_store\",\n  \"local_government_office\",\n  \"locksmith\",\n  \"lodging\",\n  \"meal_delivery\",\n  \"meal_takeaway\",\n  \"mosque\",\n  \"movie_rental\",\n  \"movie_theater\",\n  \"moving_company\",\n  \"museum\",\n  \"night_club\",\n  \"painter\",\n  \"park\",\n  \"parking\",\n  \"pet_store\",\n  \"pharmacy\",\n  \"physiotherapist\",\n  \"plumber\",\n  \"police\",\n  \"post_office\",\n  \"primary_school\",\n  \"real_estate_agency\",\n  \"restaurant\",\n  \"roofing_contractor\",\n  \"rv_park\",\n  \"school\",\n  \"secondary_school\",\n  \"shoe_store\",\n  \"shopping_mall\",\n  \"spa\",\n  \"stadium\",\n  \"storage\",\n  \"store\",\n  \"subway_station\",\n  \"supermarket\",\n  \"synagogue\",\n  \"taxi_stand\",\n  \"tourist_attraction\",\n  \"train_station\",\n  \"transit_station\",\n  \"travel_agency\",\n  \"university\",\n  \"veterinary_care\",\n  \"zoo\"\n]\n\n"

Inicialização do modelo

In [20]:
model = genai.GenerativeModel(model_name=model_name,
                              generation_config=generation_config,
                              system_instruction=prompt,
                              safety_settings=safety_settings)

Inicia chat

In [21]:
chat = model.start_chat(history=[])

Executa a busca na api Places do google, para isso criei uma funcao no google cloud

In [22]:
def search_places(gemini_response):
  url = 'https://us-central1-botlistae-teste.cloudfunctions.net/search_places_for_gemini'
  response = requests.post(url, json=gemini_response)
  if response.status_code == 200:
    return response.json()
  else:
    return None

Apresenta resposta após consulta na API places

In [23]:
def print_places(places):
  for place in places:
    print("########################")
    print(place.get('nome', None))
    print(place.get('telefone', None))
    print(place.get('endereco', None))
    print(place.get('url_gmaps', None))
    print(place.get('site', None))

Avalia se deve chamar places ou continuar conversa

In [24]:
def should_search_places(gemini_response):
  # Verificar se 'buscas' está preenchido
  if 'buscas' in gemini_response['response'] and gemini_response['response']['buscas'] and 'endereco' in gemini_response['response'] and gemini_response['response']['endereco']:
    return True
  else:
    return False


In [ ]:
resposta = input("Esperando resposta: ")
try:
  while resposta != "fim":
    response_string = chat.send_message(resposta).text
    padrao = r'{"response":{.*?}}'
    resultados = re.findall(padrao, response_string)
    if resultados:
      json_encontrado = resultados[0]
      response = json.loads(json_encontrado)
      if should_search_places(response):
        places = search_places(response)
        print_places(places)
        break
      else:
        print("Iamiga responde: ", response['response'].get('proxima_mensagem', None))
        resposta = input("Esperando resposta: ")
except json.JSONDecodeError as e:
  print("Erro ao decodificar JSON. Resposta inválida: ", e)